In [238]:
# for each interesting sequence of endpoint request, we can reset the database and save the whole generated sequence 
# some models provide examples which can be mutated on?

# if we get a rare result (non error?) for a request, it would be interesting to mutate that request
# switching a little on each parameter one by one 

# if we get an error it would be interesting to do shrinking

# can we do iterations with dotcover where we do multiple runs and see if we can increase code coverage
# something like execute requests (from python) -> check coverage -> change strategy -> exec reqs -> check if cov improved -> ...
# then collect a csv of best coverage requests which can be executed from dotnet test and we can see if mutation score increased

# idea: Get all usernames from leaderboard/top endpoint, then use those to generate emails, see if they exist using /account/email-exists

# arguments for having seperate grammars: 
# - easier overview/granular editing
# - worst case it is can be exactly the same as single big grammar
# - performance
# - less magic string parsing, wouldnt have to parse "POST;/api/getCoffee {body};[200, 404]"
# - same param name might mean different things for different endpoints, i.e. id
# - endpoints might have destructive effects on database (DELETE, PUT) and sequence of them being called matters
# - some endpoints need auth, others don't

# restart database (from host machine): docker exec mssql /usr/local/bin/docker-entrypoint.sh
# run server with coverage: dotnet-coverage collect -f cobertura -if bin/Debug/net8.0/CoffeeCard.Library.dll -if bin/Debug/net8.0/CoffeeCard.WebApi.dll dotnet bin/Debug/net8.0/CoffeeCard.WebApi.dll
# generate report: reportgenerator -reports:output.cobertura.xml -targetdir:/workspace/coverage-report -classfilters:"-CoffeeCard.Library.Migrations.*"

# TODO: work with different user groups

In [239]:
# coffee_host = "https://core.prd.analogio.dk"
coffee_host = "http://0.0.0.0:8080"

import requests

response_v2 = requests.get(f'{coffee_host}/swagger/v2/swagger.json')
swagger_v2 = response_v2.json()

In [ ]:
def get_auth_header(user_email, password):
	token = requests.post(
		url=f"{coffee_host}/api/v1/Account/login",
		json={ "email": user_email, "password": password, "version": "2.1.0" },
		headers={"Content-Type": "application/json"}
	).json()
	return f"Bearer {token['token']}"

# Authentication token for a customer (using impersonate password).
auth_token = get_auth_header("john@doe.com", "impersonate")

# Authentication token for a customer (using real password).
auth_token_real = get_auth_header("john@doe.com", "A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ=")

# Authentication token for a board member (using real password).
auth_token_real_board = get_auth_header("boardmember@analog.analog", "mvFbM25qlhmShTffMLLmojdlafz51+dz7M7eZWBlKaA=")

# TODO: Fuzz the authentication token itself?

In [241]:
from fuzzingbook.Grammars import Grammar, is_valid_grammar, trim_grammar, opts
from fuzzingbook.GeneratorGrammarFuzzer import GeneratorGrammarFuzzer
from collections import Counter

class Endpoint:
	def __init__(self, method: str, grammar: Grammar, response_codes: list[str]):
		self.method = method.upper()
		self.grammar = trim_grammar(grammar) # TODO: if there is only a single expansion that uses string or integer, flatten them
		assert is_valid_grammar(self.grammar)
		self.response_codes = response_codes
	
	def single_gen(self):
		fuzzer = GeneratorGrammarFuzzer(self.grammar)
		return fuzzer.fuzz()

	def to_str(self):
		return f"{self.method} {self.grammar['<start>'][0]}"
	
	def _matches_pattern(self, status_code: int, pattern: str) -> bool:
		"""Check if status code matches pattern like '5xx', '200', or 'xxx'"""
		code_str = str(status_code)
		if len(code_str) != 3 or len(pattern) != 3:
			return False
		for i in range(3):
			if pattern[i] != 'x' and pattern[i] != code_str[i]:
				return False
		return True
	
	# put cool algorithm here:
	def test(self, times: int, token = None, print_if_response_code = None):
		print(self.to_str(), "  documented response codes:", self.response_codes)
		fuzzer = GeneratorGrammarFuzzer(self.grammar)
		response_codes = []
		for i in range(times):
			call = fuzzer.fuzz()
			
			e = call.split(" ")[0]
			d = call.split("requestBody: ")[1] if "requestBody: " in call else ""

			r = requests.request(
				method=self.method,
				url=f"{coffee_host}{e}",
				data=d,
				headers={"Content-Type": "application/json", "Authorization": token}
			)
			
			if print_if_response_code and self._matches_pattern(r.status_code, print_if_response_code):
				print(self.method, call)
				print(r.status_code, r.text)
			
			response_codes.append(r.status_code)
		code_counts = Counter(response_codes)
		print("Response code counts:", '{', ", ".join([f"{pair[0]}: {pair[1]}" for pair in list(sorted([(code, count) for code, count in dict(code_counts).items()], key=lambda item: item[0]))]), '}', '\n======')

In [242]:
from itertools import combinations
import random
import numpy as np

def random_query_concat(params):
	if not params:
		return [""]
	results = [""]
	for r in range(1, len(params) + 1):
		for combo in combinations(params, r):
			results.append("?" + "&".join(combo))
	return results

def random_request_body(props, required):
	body = ""
	for prop in [p for p in props if any(p.startswith("\"" + r + "\"") for r in required)]:
		if body == "":
			body += prop
		else:
			body += ", " + prop
	results = ["{ " + body + " }"]
	non_required = [p for p in props if not any(p.startswith("\"" + r + "\"") for r in required)]
	for r in range(1, len(non_required) + 1):
		for combo in combinations(non_required, r):
			results.append("{ " + body + ("" if body == "" else ", ") + ", ".join(combo) + " }")
	return results

def random_string(min_length, max_length, avg_length=50):
	# Use geometric distribution to favor shorter strings
	p = 1 / avg_length
	length = min(np.random.geometric(p) - 1 + min_length, max_length)
	return '"' + ''.join(random.choices("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ", k=length)) + '"'

def random_email():
	return random_string(1, 10)[:-1].lower() + "@" + random_string(1, 6)[1:][:-1].lower() + ".com\""

def random_int_with_bounds(min_value, max_value):
	if random.random() < 0.1:
		return random.choice([min_value, max_value])
	return random.randint(min_value, max_value)

# sample tests for random_string
# [random_string(1, 100000, avg_length=50) for _ in range(10)]

In [243]:
# define some common primitives
MIN_INT = -(2 ** 31)
MAX_INT = 2 ** 31 - 1
rand_num = ('10', opts(pre=lambda: random_int_with_bounds(MIN_INT, MAX_INT)))
rand_pos = ('10', opts(pre=lambda: random_int_with_bounds(1, MAX_INT)))

rand_string = ('"random"', opts(pre=lambda: random_string(1, 100000)))
rand_email = ('"john@doe.com"', opts(pre=lambda: random_email()))
rand_date = ('"01-01-2022"', opts(pre=lambda: f"{str(random.randint(1, 30)).zfill(2)}-{str(random.randint(1, 12)).zfill(2)}-{random.randint(2018, 2028)}"))

def addBasePrimitives(grammar: Grammar): 
	grammar["<boolean>"] = ["true", "false"]
	grammar["<string>"] = ["\"\"", rand_string]
	grammar["<integer>"] = ["1", "0", "-1", rand_num]

In [244]:
def addModels(grammar: Grammar, primitives: set, swagger_doc):
	for modelName, schema in swagger_doc["components"]["schemas"].items():
		if "type" in schema:
			if schema["type"] == "object":
				props = []
				for param_name, param_schema in schema["properties"].items():
					param_type = ""
					if "type" in param_schema:
						param_type = param_schema["type"]
						if param_type == "array":
							assert "items" in param_schema
							item_type = ""
							if "type" in param_schema["items"]:
								item_type = param_schema["items"]["type"]
								primitives.add(item_type)
							elif "$ref" in param_schema["items"]:
								item_type = param_schema["items"]["$ref"].split("/")[-1] 
							assert not item_type == ""
							param_type += "." + item_type
							grammar[f"<{param_type}>"] = [f"<{item_type}>", f"<{item_type}>, <{param_type}>"]
						else:
							primitives.add(param_type)
					elif "oneOf" in param_schema:
						param_type = param_schema["oneOf"][0]["$ref"].split("/")[-1]
					assert not param_type == ""
					if param_type == "boolean":
						param_expansion = f"<{param_type}>"
					else:
						param_expansion = f"<{param_name}.{param_type}>"
					props.append(f"\"{param_name}\": {param_expansion}")
					if param_type.startswith("array."):
						grammar[param_expansion] = ["[]", f"[<{param_type}>]"]
					else:
						grammar[param_expansion] = [f"<{param_type}>"]
					if "example" in param_schema and not str(param_schema["example"]) == "[no example provided]":
						grammar[param_expansion].append(repr(param_schema["example"]).replace("\'", "\""))
				grammar[f"<{modelName}>"] = random_request_body(props, [] if not "required" in schema else schema["required"])
			elif "enum" in schema:
				grammar[f"<{modelName}>"] = ["\"" + m + "\"" for m in schema["enum"]]
		
	# elif "allOf" in schema:  TODO: <<<

In [245]:
def get_endpoints(swagger_doc):
	endpoints = []
	for path, reqs in swagger_doc["paths"].items(): # iterate through endpoints
		for req_method, details in reqs.items(): # same path may have endpoint defined for multiple methods
			primitives = set() # primitive typpes used by the request
			grammar = {}
			start_expansion = path
			query_strings = [] # collect query params
			if ("parameters" in details and details["parameters"]):
				for param in details["parameters"]:
					param_name = param["name"]
					param_type = ""
					if "type" in param["schema"]:
						param_type = param["schema"]["type"]
						primitives.add(param_type)
					elif "oneOf" in param["schema"]:
						param_type = param["schema"]["oneOf"][0]["$ref"].split("/")[-1]
					assert not param_type == ""
					if param_type == "boolean":
						param_expansion = f"<{param_type}>"
					else:
						param_expansion = f"<{param_name}.{param_type}>"
					assert param["in"] == "path" or param["in"] == "query"
					if param["in"] == "path":
						start_expansion = start_expansion.replace("{" + param_name + "}", param_expansion) # insert expansion pattern in path
					if param["in"] == "query":
						query = f"{param_name}={param_expansion}"
						query_strings.append(query)
					grammar[param_expansion] = [f"<{param_type}>"]
			if not len(query_strings) == 0:
				query_expansion = f"<({path}.{req_method})-query>"
				start_expansion = f"{start_expansion}{query_expansion}" # if there are query params at expansion to path in start expansion
			if ("requestBody" in details and details["requestBody"]):
				model = details["requestBody"]["content"]["application/json"]["schema"]["$ref"].split("/")[-1]
				start_expansion += f" requestBody: <{model}>" # if there is request body add to start expansion
			grammar["<start>"] = [start_expansion]
			if not len(query_strings) == 0:
				grammar[query_expansion] = random_query_concat(query_strings)
			addModels(grammar, primitives, swagger_doc)
			addBasePrimitives(grammar)
			endpoint = Endpoint(req_method, grammar, [k for k in details["responses"].keys()])
			endpoints.append(endpoint)
	return endpoints


In [246]:
endpoints_v2 = get_endpoints(swagger_v2)

In [247]:
endpoint = endpoints_v2[0]
endpoint.grammar = {
	'<start>': ['/api/v2/account requestBody: <RegisterAccountRequest>'],
	'<RegisterAccountRequest>': ['{ "name": <name.string>, "email": <email.string>, "password": <password.string>, "programmeId": <programmeId.integer> }'],
	'<name.string>': ['<string>', '"Espresso"'],
	'<email.string>': ['<string>', '"john@doe.com"', ('"lukas@swag.com"', opts(pre=random_email))],
	'<password.string>': ['<string>'],
	'<string>': ['"lukas"', '"swagg"', '""'],
	'<programmeId.integer>': ['21', '0', '-1'],
}

endpoint.test(100)

POST /api/v2/account requestBody: <RegisterAccountRequest>   documented response codes: ['201', '409']
Response code counts: { 201: 7, 400: 72, 409: 21 } 


In [248]:
endpoint = endpoints_v2[1]

# should deleting a deleted account not return error?
# TODO: how to trigger 429 response?
endpoint.test(2)
endpoint.test(2, get_auth_header("tester@analogio.dk")) 

# TODO: test verify link: (send with email, obtain right token somehow)
requests.get(
	url=f"{coffee_host}/verifydelete?token={get_auth_header('tester@analogio.dk')[7:]}"
).text

DELETE /api/v2/account   documented response codes: ['202', '401', '429']
Response code counts: { 401: 2 } 
DELETE /api/v2/account   documented response codes: ['202', '401', '429']
Response code counts: { 202: 2 } 


'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <title>Cafe Analog - Result of request</title>\n    <link rel="stylesheet" href="/css/site.css" />\n</head>\n\n<body>\n<div class="box">\n    <div class="row header">\n        <img id="logo" src="/images/AnalogLogo.svg" alt="Analog logo" />\n    </div>\n    <div class="row content">\n        <div class="wrapper">\n            \n<div id="results">\n        <h1>Error</h1>\n        <p>Looks like the link you used has expired or already been used. Request a new link and try again</p>\n</div>\n        </div>\n    </div>\n    <div class="row footer">\n        <p>\n            <strong>Cafe Analog</strong><br/>\n            Rued Langgaards Vej 7, 2300 Copenhagen S / CVR: 34657343<br/>\n            <a href="mailto:support@analogio.dk">support@analogio.dk</a> / <a href="https://www.cafeanalog.dk">www.cafeanalog.dk</a>\n        </p>\n    </div>\n</div>\n\n<script src="https://code.jquery.com/jquery-3.5.0.min.js" runat="server"></script>\n<script src

In [249]:
endpoint = endpoints_v2[2]

endpoint.test(2)
endpoint.test(2, auth_token)

GET /api/v2/account   documented response codes: ['200', '401']
Response code counts: { 401: 2 } 
GET /api/v2/account   documented response codes: ['200', '401']
Response code counts: { 200: 2 } 


In [250]:
endpoint = endpoints_v2[3]
endpoint.grammar = {
	'<start>': ['/api/v2/account requestBody: <UpdateUserRequest>'],
	'<name.string>': ['<string>', '"Espresso"'],
	'<email.string>': ['<string>', '"john@doe.com"'],
	'<password.string>': ['<string>'],
	'<programmeId.integer>': ['<integer>', '1'],
	'<boolean>': ['true', 'false'],
	'<UpdateUserRequest>': ['{  }',
	'{ "name": <name.string> }',
	'{ "email": <email.string> }',
	'{ "privacyActivated": <boolean> }',
	'{ "programmeId": <programmeId.integer> }',
	'{ "password": <password.string> }',
	'{ "name": <name.string>, "email": <email.string> }',
	'{ "name": <name.string>, "privacyActivated": <boolean> }',
	'{ "name": <name.string>, "programmeId": <programmeId.integer> }',
	'{ "name": <name.string>, "password": <password.string> }',
	'{ "email": <email.string>, "privacyActivated": <boolean> }',
	'{ "email": <email.string>, "programmeId": <programmeId.integer> }',
	'{ "email": <email.string>, "password": <password.string> }',
	'{ "privacyActivated": <boolean>, "programmeId": <programmeId.integer> }',
	'{ "privacyActivated": <boolean>, "password": <password.string> }',
	'{ "programmeId": <programmeId.integer>, "password": <password.string> }',
	'{ "name": <name.string>, "email": <email.string>, "privacyActivated": <boolean> }',
	'{ "name": <name.string>, "email": <email.string>, "programmeId": <programmeId.integer> }',
	'{ "name": <name.string>, "email": <email.string>, "password": <password.string> }',
	'{ "name": <name.string>, "privacyActivated": <boolean>, "programmeId": <programmeId.integer> }',
	'{ "name": <name.string>, "privacyActivated": <boolean>, "password": <password.string> }',
	'{ "name": <name.string>, "programmeId": <programmeId.integer>, "password": <password.string> }',
	'{ "email": <email.string>, "privacyActivated": <boolean>, "programmeId": <programmeId.integer> }',
	'{ "email": <email.string>, "privacyActivated": <boolean>, "password": <password.string> }',
	'{ "email": <email.string>, "programmeId": <programmeId.integer>, "password": <password.string> }',
	'{ "privacyActivated": <boolean>, "programmeId": <programmeId.integer>, "password": <password.string> }',
	'{ "name": <name.string>, "email": <email.string>, "privacyActivated": <boolean>, "programmeId": <programmeId.integer> }',
	'{ "name": <name.string>, "email": <email.string>, "privacyActivated": <boolean>, "password": <password.string> }',
	'{ "name": <name.string>, "email": <email.string>, "programmeId": <programmeId.integer>, "password": <password.string> }',
	'{ "name": <name.string>, "privacyActivated": <boolean>, "programmeId": <programmeId.integer>, "password": <password.string> }',
	'{ "email": <email.string>, "privacyActivated": <boolean>, "programmeId": <programmeId.integer>, "password": <password.string> }',
	'{ "name": <name.string>, "email": <email.string>, "privacyActivated": <boolean>, "programmeId": <programmeId.integer>, "password": <password.string> }'],
	'<string>': ['"lukas"', '"swagg"', '""'],
	'<integer>': ['1', '0', '-1']
}

endpoint.test(2)
endpoint.test(1000, get_auth_header("testme@mt2015.com"))

PUT /api/v2/account requestBody: <UpdateUserRequest>   documented response codes: ['200', '401']
Response code counts: { 401: 2 } 
PUT /api/v2/account requestBody: <UpdateUserRequest>   documented response codes: ['200', '401']
Response code counts: { 200: 406, 400: 594 } 


In [251]:
# POST /api/v2/account/email-exists
endpoint = endpoints_v2[4]
endpoint.grammar = {
	'<start>': ['/api/v2/account/email-exists requestBody: <EmailExistsRequest>'],
 	'<EmailExistsRequest>': ['{ "email": <email.string> }'],
 	'<email.string>': [rand_string, rand_email, '""', '"john@doe.com"', '"swag@god.com"']
}

endpoint.test(10)
endpoint.test(100, auth_token, print_if_response_code="4xx")

POST /api/v2/account/email-exists requestBody: <EmailExistsRequest>   documented response codes: ['200', '401']
Response code counts: { 200: 8, 400: 2 } 
POST /api/v2/account/email-exists requestBody: <EmailExistsRequest>   documented response codes: ['200', '401']
POST /api/v2/account/email-exists requestBody: { "email": "" }
400 {"type":"https://tools.ietf.org/html/rfc9110#section-15.5.1","title":"One or more validation errors occurred.","status":400,"errors":{"Email":["The Email field is required.","The Email field is not a valid e-mail address."]},"traceId":"00-a6807ceea6102a8c7571b78dd98d83b6-78b304894d5e73d9-01"}
POST /api/v2/account/email-exists requestBody: { "email": "igH" }
400 {"type":"https://tools.ietf.org/html/rfc9110#section-15.5.1","title":"One or more validation errors occurred.","status":400,"errors":{"Email":["The Email field is not a valid e-mail address."]},"traceId":"00-14a4217fcdc3f2d6a6f9bd32afac5e35-726344b085ee7ef3-01"}
POST /api/v2/account/email-exists reques

In [252]:
endpoint = endpoints_v2[5]
endpoint.grammar = {
	'<start>': ['/api/v2/account/<id.integer>/user-group requestBody: <UpdateUserGroupRequest>'],
	'<userGroup.UserGroup>': ['<UserGroup>', '"Barista"'],
	'<UpdateUserGroupRequest>': ['{ "userGroup": <userGroup.UserGroup> }'],
	'<UserGroup>': ['"Customer"', '"Barista"', '"Manager"', '"Board"'],
	'<id.integer>': ['<integer>', '122'],
	'<integer>': ['0', '-1', '1', '5', '10']
} 
endpoint.test(10, get_auth_header("testme@mt2015.com")) # user group: customer
endpoint.test(100, auth_token) # user group: board member

PATCH /api/v2/account/<id.integer>/user-group requestBody: <UpdateUserGroupRequest>   documented response codes: ['204', '401', '404']
Response code counts: { 403: 10 } 
PATCH /api/v2/account/<id.integer>/user-group requestBody: <UpdateUserGroupRequest>   documented response codes: ['204', '401', '404']
Response code counts: { 204: 24, 404: 76 } 


In [253]:
endpoint = endpoints_v2[6]
endpoint.grammar = {
  	'<start>': ['/api/v2/account/resend-verification-email requestBody: <ResendAccountVerificationEmailRequest>'],
	'<ResendAccountVerificationEmailRequest>': ['{ "email": <email.string> }'],
	'<email.string>': ['<string>', '"john@doe.com"', '"Sgddhh@mt2015.Com"'],
	'<string>': ['"lukas"', '"swagg"', '""']
}
endpoint.test(100, auth_token)

POST /api/v2/account/resend-verification-email requestBody: <ResendAccountVerificationEmailRequest>   documented response codes: ['200', '404', '409']
Response code counts: { 200: 36, 400: 35, 409: 29 } 


In [254]:
endpoint = endpoints_v2[7]
endpoint.grammar = {
	'<start>': ['/api/v2/account/search<(/api/v2/account/search.get)-query>'],
	'<(/api/v2/account/search.get)-query>': [
    	'',
		'?pageNum=<pageNum.integer>',
		'?filter=<filter.string>',
		'?pageLength=<pageLength.integer>',
		'?pageNum=<pageNum.integer>&filter=<filter.string>',
		'?pageNum=<pageNum.integer>&pageLength=<pageLength.integer>',
		'?filter=<filter.string>&pageLength=<pageLength.integer>',
		'?pageNum=<pageNum.integer>&filter=<filter.string>&pageLength=<pageLength.integer>'],
   	'<pageNum.integer>': ['<integer>'],
	'<filter.string>': ['<string>', '5', '10', 'john@doe.com', 'string', 'Omid Sabihi Marfavi'],
	'<pageLength.integer>': ['<integer>'],
	'<string>': ['""', ('"random"', opts(pre=lambda: random_string(1, 100000)))],
	'<integer>': ['1', '0', '-1', ('10', opts(pre=lambda: random.randint(0, 2147483647)))]
}

endpoint.test(10)
endpoint.test(400, auth_token)

GET /api/v2/account/search<(/api/v2/account/search.get)-query>   documented response codes: ['401', '200']
Response code counts: { 401: 10 } 
GET /api/v2/account/search<(/api/v2/account/search.get)-query>   documented response codes: ['401', '200']
Response code counts: { 200: 183, 400: 206, 500: 11 } 


In [255]:
endpoint = endpoints_v2[8]
endpoint.grammar = {
    '<start>': ['/api/v2/account/login requestBody: <UserLoginRequest>'],
    '<UserLoginRequest>': ['{ "email": <email.string>, "loginType": <loginType.LoginType> }'],
    '<loginType.LoginType>': ['<LoginType>', '"Shifty"'],
    '<LoginType>': ['"Shifty"', '"App"'],
    '<email.string>': ['<string>', '"john@doe.com"', rand_email],
    '<string>': ['""', rand_string]
}
endpoint.test(100)

# What we found: 500 errors if the login type is "App" AND the email is already registered

POST /api/v2/account/login requestBody: <UserLoginRequest>   documented response codes: ['204', 'default']
Response code counts: { 204: 60, 400: 35, 500: 5 } 


In [256]:
endpoint = endpoints_v2[9]
endpoint.grammar = {
	'<start>': ['/api/v2/account/auth requestBody: <TokenLoginRequest>'],
	'<TokenLoginRequest>': ['{ "token": <token.string> }'],
	'<token.string>': ['"d57ed472-bf06-436e-96b7-aed7690ed898"', rand_string, '""', 'null'],
    # NOTE: Cannot get valid tokens easily.
    # We get the token from dotnet output
}
endpoint.test(100)

POST /api/v2/account/auth requestBody: <TokenLoginRequest>   documented response codes: ['200', '404', 'default']
Response code counts: { 400: 51, 401: 49 } 


In [257]:
endpoint = endpoints_v2[10]
endpoint.grammar = {
	'<start>': ['/api/v2/statistics/unused-clips requestBody: <UnusedClipsRequest>'],
	'<UnusedClipsRequest>': ['{  }',
		'{ "startDate": <startDate.string> }',
		'{ "endDate": <endDate.string> }',
		'{ "startDate": <startDate.string>, "endDate": <endDate.string> }'],
	'<startDate.string>': ['<string>', '"2021-02-08"', rand_date],
	'<endDate.string>': ['<string>', '"2024-02-08"', rand_date],
	'<string>': ['""', rand_string]
}

endpoint.test(100, auth_token)

POST /api/v2/statistics/unused-clips requestBody: <UnusedClipsRequest>   documented response codes: ['200', '401']
Response code counts: { 200: 3, 400: 97 } 


In [258]:
endpoint = endpoints_v2[11]
endpoint.test(2)

GET /api/v2/appconfig   documented response codes: ['200']
Response code counts: { 200: 2 } 


In [259]:
endpoint = endpoints_v2[12]
endpoint.test(10, "X-API-Key local-development-apikey")
endpoint = endpoints_v2[13]
endpoint.test(10, "X-API-Key local-development-apikey")

GET /api/v2/health/ping   documented response codes: ['200']
Response code counts: { 200: 10 } 
GET /api/v2/health/check   documented response codes: ['200', '503', 'default']
Response code counts: { 200: 10 } 


In [260]:
endpoint = endpoints_v2[14]
endpoint.test(100, auth_token)

GET /api/v2/leaderboard/top<(/api/v2/leaderboard/top.get)-query>   documented response codes: ['200']
Response code counts: { 200: 34, 400: 61, 500: 5 } 


In [261]:
endpoint = endpoints_v2[15]
endpoint.test(100, auth_token)
endpoint.test(1) # trigger 401

GET /api/v2/leaderboard<(/api/v2/leaderboard.get)-query>   documented response codes: ['200', '401']
Response code counts: { 200: 49, 400: 51 } 
GET /api/v2/leaderboard<(/api/v2/leaderboard.get)-query>   documented response codes: ['200', '401']
Response code counts: { 401: 1 } 


In [262]:
endpoint = endpoints_v2[16]
endpoint.test(10, auth_token)

GET /api/v2/menuitems   documented response codes: ['200']
Response code counts: { 200: 10 } 


In [263]:
endpoint = endpoints_v2[17]
endpoint.grammar = {'<start>': ['/api/v2/menuitems requestBody: <AddMenuItemRequest>'],
 '<AddMenuItemRequest>': ['{ "name": <name.string> }'],
 '<name.string>': ['<string>', '"Espresso"', ("item", opts(pre=lambda: random_string(1, 20)))],
 '<string>': ['""', rand_string]}
endpoint.test(100, auth_token)

POST /api/v2/menuitems requestBody: <AddMenuItemRequest>   documented response codes: ['201']
Response code counts: { 201: 49, 400: 14, 409: 37 } 


In [264]:
# PUT /api/v2/menuitems/{id}
endpoint = endpoints_v2[18]
endpoint.grammar = {
	'<start>': ['/api/v2/menuitems/<id.integer> requestBody: <UpdateMenuItemRequest>'],
	'<UpdateMenuItemRequest>': ['{ "name": <name.string>, "active": <boolean> }'],
	'<boolean>': ['true', 'false'],
	'<id.integer>': ['7', '17', '0', ('0', opts(pre=lambda: random.randint(-2147483648, 2147483647)))],
	'<name.string>': ['""', '"Espresso"'],
}

endpoint.test(10)
endpoint.test(100, auth_token)

PUT /api/v2/menuitems/<id.integer> requestBody: <UpdateMenuItemRequest>   documented response codes: ['200']
Response code counts: { 401: 10 } 
PUT /api/v2/menuitems/<id.integer> requestBody: <UpdateMenuItemRequest>   documented response codes: ['200']
Response code counts: { 200: 9, 400: 42, 404: 24, 409: 25 } 


In [265]:
endpoint = endpoints_v2[19]
endpoint.grammar = {
	'<start>': ['/api/v2/products requestBody: <AddProductRequest>'],
	'<AddProductRequest>': ['{ "price": <price.integer>, "numberOfTickets": <numberOfTickets.integer>, "name": <name.string>, "description": <description.string>, "visible": <boolean>, "allowedUserGroups": <allowedUserGroups.array.UserGroup>, "menuItemIds": <menuItemIds.array.integer> }'],
	'<UserGroup>': ['"Customer"', '"Barista"', '"Manager"', '"Board"'],
	'<name.string>': ['<string>', '"Espresso"'],
	'<boolean>': ['true', 'false'],
	'<price.integer>': ['<integer>', '10'],
	'<numberOfTickets.integer>': ['<integer>', '10'],
	'<description.string>': ['<string>', '"Voucher codes for intro week   "'],
	'<array.UserGroup>': ['<UserGroup>', '<UserGroup>, <array.UserGroup>'],
	'<allowedUserGroups.array.UserGroup>': ['[]',
		'[<array.UserGroup>]',
		'["Manager", "Board"]'],
	'<array.integer>': ['<integer>', '<integer>, <array.integer>'],
	'<menuItemIds.array.integer>': ['[]', '[<array.integer>]', '[1, 2]'],
	'<string>': ['""', rand_string],
	'<integer>': ['1', '0', '-1', rand_num]
}
endpoint.test(10)
endpoint.test(100, auth_token)

POST /api/v2/products requestBody: <AddProductRequest>   documented response codes: ['201']
Response code counts: { 401: 10 } 
POST /api/v2/products requestBody: <AddProductRequest>   documented response codes: ['201']
Response code counts: { 201: 18, 400: 62, 409: 19, 500: 1 } 


In [266]:
endpoint = endpoints_v2[20]
endpoint.test(10)
endpoint.test(10, auth_token)

GET /api/v2/products   documented response codes: ['200', '401']
Response code counts: { 401: 10 } 
GET /api/v2/products   documented response codes: ['200', '401']
Response code counts: { 200: 10 } 


In [267]:
endpoint = endpoints_v2[21]
endpoint.grammar = {
	'<start>': ['/api/v2/products/<id.integer> requestBody: <UpdateProductRequest>'],
	'<UpdateProductRequest>': ['{ "price": <price.integer>, "numberOfTickets": <numberOfTickets.integer>, "name": <name.string>, "description": <description.string>, "visible": <boolean>, "allowedUserGroups": <allowedUserGroups.array.UserGroup>, "menuItemIds": <menuItemIds.array.integer> }'],
	'<allowedUserGroups.array.UserGroup>': ['[]',
		'[<array.UserGroup>]',
		'["Manager", "Board"]'],
	'<id.integer>': ['<integer>', '122'],
	'<name.string>': ['<string>', '"Espresso"'],
	'<boolean>': ['true', 'false'],
	'<UserGroup>': ['"Customer"', '"Barista"', '"Manager"', '"Board"'],
	'<price.integer>': ['<integer>', '10'],
	'<numberOfTickets.integer>': ['<integer>', '10'],
	'<description.string>': ['<string>', '"Voucher codes for intro week   "'],
	'<array.UserGroup>': ['<UserGroup>', '<UserGroup>, <array.UserGroup>'],
	'<array.integer>': ['<integer>', '<integer>, <array.integer>'],
	'<menuItemIds.array.integer>': ['[]', '[<array.integer>]', '[1, 2]'],
	'<string>': ['""', rand_string],
	'<integer>': ['1', '0', '-1', rand_num]
}

endpoint.test(100, auth_token)

PUT /api/v2/products/<id.integer> requestBody: <UpdateProductRequest>   documented response codes: ['200']
Response code counts: { 200: 3, 400: 67, 500: 30 } 


In [268]:
endpoint = endpoints_v2[22]
endpoint.test(100, auth_token)

GET /api/v2/products/<id.integer>   documented response codes: ['200', '401', '404']
Response code counts: { 200: 12, 404: 88 } 


In [269]:
endpoint = endpoints_v2[23]
endpoint.test(10, auth_token)

GET /api/v2/products/all   documented response codes: ['200']
Response code counts: { 200: 10 } 


In [270]:
# endpoints[24..28] /purchases
endpoint = endpoints_v2[24]
endpoint.test(100, auth_token)
endpoint = endpoints_v2[25]
endpoint.test(100, auth_token)
endpoint = endpoints_v2[26]
endpoint.test(100, auth_token)
endpoint = endpoints_v2[27]
endpoint.test(100, auth_token)
endpoint = endpoints_v2[28]
endpoint.test(100, auth_token)

GET /api/v2/purchases   documented response codes: ['200', '401', 'default']
Response code counts: { 200: 100 } 
POST /api/v2/purchases requestBody: <InitiatePurchaseRequest>   documented response codes: ['200', '401', '403']
Response code counts: { 400: 3, 403: 50, 404: 35, 500: 12 } 
GET /api/v2/purchases/user/<userId.integer>   documented response codes: ['200', '401', '403', '404']
Response code counts: { 404: 100 } 
GET /api/v2/purchases/<id.integer>   documented response codes: ['200', '401', '404']
Response code counts: { 404: 100 } 
PUT /api/v2/purchases/<id.integer>/refund   documented response codes: ['200', '401', '403']
Response code counts: { 403: 58, 404: 42 } 


In [271]:
endpoint = endpoints_v2[29]
endpoint.test(100, auth_token)

GET /api/v2/tickets<(/api/v2/tickets.get)-query>   documented response codes: ['200', '401']
Response code counts: { 200: 100 } 


In [272]:
endpoint = endpoints_v2[30]
endpoint.grammar = {
	'<start>': ['/api/v2/tickets/use requestBody: <UseTicketRequest>'],
	'<productId.integer>': ['<integer>', '6'],
	'<menuItemId.integer>': ['<integer>', '1'],
	'<UseTicketRequest>': ['{ "productId": <productId.integer>, "menuItemId": <menuItemId.integer> }'],
	'<integer>': ['1', '0', '-1', ('10', opts(pre=lambda: random_int_with_bounds(1, 100)))]
}
endpoint.test(100, get_auth_header("test@analogio.dk"))
# TODO: find and use user that has tickets! produce 200

POST /api/v2/tickets/use requestBody: <UseTicketRequest>   documented response codes: ['200', '401', '403', '404']
Response code counts: { 403: 54, 404: 46 } 


In [273]:
# endpoint = endpoints_v2[31]
# endpoint.test(1, auth_token) # FIXME: this time out!
# 500 maybe because of mock


In [274]:
endpoint = endpoints_v2[32]
endpoint.grammar = {
	'<start>': ['/api/v2/vouchers/<voucher-code.string>/redeem'],
	'<voucher-code.string>': ['<string>',
		"ESP-<string>",
        # valid unredeemed vouchers
		"ESP-T4JA2U",
        "ESP-BFRRIS",
        "ESP-VYKZ1C"
	],
	'<string>': ['""', rand_string, ('"random"', opts(pre=lambda: random_string(6, 6).upper()))]
}

endpoint.test(100, auth_token)
# NOTE: When run, the unredeemed vouchers become redeemed,
# so only the first few run(s) will get 200 responses

POST /api/v2/vouchers/<voucher-code.string>/redeem   documented response codes: ['200', '400', '401', '404']
Response code counts: { 200: 3, 404: 41, 409: 56 } 


In [275]:
endpoint = endpoints_v2[33]
endpoint.test(100, "X-API-Key local-development-apikey") 

PUT /api/v2/webhooks/accounts/user-group requestBody: <WebhookUpdateUserGroupRequest>   documented response codes: ['204', '401', '400']
Response code counts: { 204: 100 } 


# V1 endpoints

In [276]:
response_v1 = requests.get(f'{coffee_host}/swagger/v1/swagger.json')
swagger_v1 = response_v1.json()

endpoints_v1 = get_endpoints(swagger_v1)

In [277]:
# v1/Account/register

endpoint = endpoints_v1[0]
endpoint.grammar = {
    '<start>': ['/api/v1/Account/register requestBody: <RegisterDto>'],
    '<name.string>': ['<string>', '"Coffee clip card"'],
    '<email.string>': ['<string>', '"john@doe.com"', rand_email],
    '<password.string>': ['<string>'],
    '<RegisterDto>': ['{ "name": <name.string>, "email": <email.string>, "password": <password.string> }'],
    '<string>': ['""', rand_string]
}

endpoint.test(100)

POST /api/v1/Account/register requestBody: <RegisterDto>   documented response codes: ['201', '409']
Response code counts: { 201: 13, 400: 74, 409: 13 } 


In [278]:
# v1/Account/login

endpoint = endpoints_v1[1]
endpoint.grammar = {
    '<start>': ['/api/v1/Account/login requestBody: <LoginDto>'],
    '<email.string>': ['<string>', '"tester@analogio.dk"', rand_email],
    '<password.string>': ['<string>', '"iI3yWuNXckJKVgxxUqHeeURA4Opc/uYoKDM6RWpQbgU="'],
    '<version.string>': ['<string>', '"2.1.0"'],
    '<LoginDto>': ['{ "email": <email.string>, "password": <password.string>, "version": <version.string> }'],
    '<string>': ['""', rand_string]
}

endpoint.test(100)

# Force 429 (rate limit)

endpoint = endpoints_v1[1]
endpoint.grammar = {
    '<start>': ['/api/v1/Account/login requestBody: <LoginDto>'],
    '<email.string>': ['"john@doe.com"'],
    '<password.string>': ['"wrong"'],
    '<version.string>': ['"2.1.0"'],
    '<LoginDto>': ['{ "email": <email.string>, "password": <password.string>, "version": <version.string> }'],
}
endpoint.test(100)

POST /api/v1/Account/login requestBody: <LoginDto>   documented response codes: ['200', '401', '403', '429']
Response code counts: { 200: 6, 400: 74, 401: 20 } 
POST /api/v1/Account/login requestBody: <LoginDto>   documented response codes: ['200', '401', '403', '429']
Response code counts: { 401: 20, 429: 80 } 


In [279]:
# GET v1/Account

endpoint = endpoints_v1[2]
endpoint.test(1, auth_token)

GET /api/v1/Account   documented response codes: ['410']
Response code counts: { 410: 1 } 


In [280]:
# PUT v1/Account

endpoint = endpoints_v1[3]
endpoint.test(1, auth_token)

PUT /api/v1/Account requestBody: <UpdateUserDto>   documented response codes: ['410']
Response code counts: { 410: 1 } 


In [281]:
# POST v1/Account/forgotpassword

endpoint = endpoints_v1[4]
endpoint.grammar = {
    '<start>': ['/api/v1/Account/forgotpassword requestBody: <EmailDto>'],
    '<email.string>': ['<string>', '"john@doe.com"', rand_email],
    '<EmailDto>': ['{ "email": <email.string> }', '{ }'],
    '<string>': ['""', rand_string]
}
endpoint.test(100)

POST /api/v1/Account/forgotpassword requestBody: <EmailDto>   documented response codes: ['200', '404']
Response code counts: { 200: 16, 400: 64, 401: 20 } 


In [282]:
# v1/AppConfig

endpoint = endpoints_v1[5]
endpoint.test(1, auth_token)

GET /api/v1/AppConfig   documented response codes: ['410']
Response code counts: { 410: 1 } 


In [283]:
# GET /api/v1/CoffeeCards

endpoint = endpoints_v1[6]
endpoint.test(1)             # 401
endpoint.test(1, auth_token) # 200

GET /api/v1/CoffeeCards   documented response codes: ['200', '401']
Response code counts: { 401: 1 } 
GET /api/v1/CoffeeCards   documented response codes: ['200', '401']
Response code counts: { 200: 1 } 


In [284]:
# GET /api/v1/Leaderboard

endpoint = endpoints_v1[7]
endpoint.grammar = {
    '<preset.integer>': ['<integer>'],
    '<top.integer>': ['<integer>'],
    '<start>': ['/api/v1/Leaderboard<(/api/v1/Leaderboard.get)-query>'],
    '<(/api/v1/Leaderboard.get)-query>': ['',
    '?preset=<preset.integer>',
    '?top=<top.integer>',
    '?preset=<preset.integer>&top=<top.integer>'],
    '<integer>': ['1', '0', '-1', rand_pos]
}
endpoint.test(10)

GET /api/v1/Leaderboard<(/api/v1/Leaderboard.get)-query>   documented response codes: ['410']
Response code counts: { 410: 10 } 


In [285]:
# POST /api/v1/MobilePay/initiate

endpoint = endpoints_v1[8]
endpoint.test(1, auth_token)

POST /api/v1/MobilePay/initiate requestBody: <InitiatePurchaseDto>   documented response codes: ['410']
Response code counts: { 410: 1 } 


In [286]:
# POST /api/v1/MobilePay/complete

endpoint = endpoints_v1[9]
endpoint.test(100, auth_token)

POST /api/v1/MobilePay/complete requestBody: <CompletePurchaseDto>   documented response codes: ['410']
Response code counts: { 400: 81, 410: 19 } 


In [287]:
# v1/Ping DEPRECATED
endpoint = endpoints_v1[10]
endpoint.test(10)

GET /api/v1/Ping   documented response codes: ['200']
Response code counts: { 500: 10 } 


In [288]:
# v1/Products
endpoint = endpoints_v1[11]
endpoint.test(10)
endpoint = endpoints_v1[12]
endpoint.test(10, auth_token)

GET /api/v1/Products   documented response codes: ['200']


Response code counts: { 200: 10 } 
GET /api/v1/Products/app   documented response codes: ['200', '401']
Response code counts: { 200: 10 } 


In [289]:
# v1/Programmes
endpoint = endpoints_v1[13]
endpoint.test(10)

GET /api/v1/Programmes   documented response codes: ['200']
Response code counts: { 200: 10 } 


In [290]:
# v1/Purchases
endpoint = endpoints_v1[14]
endpoint.test(100, auth_token)

GET /api/v1/Purchases   documented response codes: ['410']
Response code counts: { 410: 100 } 


In [291]:
# v1/Purchases/redeemvoucher
endpoint = endpoints_v1[15]
endpoint.grammar = {
	'<start>': ['/api/v1/Purchases/redeemvoucher<(/api/v1/Purchases/redeemvoucher.post)-query>'],
	'<(/api/v1/Purchases/redeemvoucher.post)-query>': ['', '?voucherCode=<voucherCode.string>'],
	'<voucherCode.string>': ['<string>', 
        "ESP-VYKZ1C",
        "ESP-60OW8R",
        "AZ2MZH"],
	'<string>': ['""', rand_string]
}
endpoint.test(100, auth_token)

POST /api/v1/Purchases/redeemvoucher<(/api/v1/Purchases/redeemvoucher.post)-query>   documented response codes: ['200', '409', '401', '404']
Response code counts: { 200: 2, 404: 55, 409: 43 } 


In [292]:
# v1/Purchases/issueproduct 
endpoint = endpoints_v1[16]
endpoint.grammar = {
	'<start>': ['/api/v1/Purchases/issueproduct requestBody: <IssueProductDto>'],
	'<IssueProductDto>': ['{ "issuedBy": <issuedBy.string>, "userId": <userId.integer>, "productId": <productId.integer> }'],
	'<productId.integer>': ['<integer>', '1'],
	'<issuedBy.string>': ['<string>'],
	'<userId.integer>': ['<integer>', '122'],
	'<string>': ['""', rand_string],
	'<integer>': ['1', '0', '-1', rand_num]
}
endpoint.test(100, auth_token)

POST /api/v1/Purchases/issueproduct requestBody: <IssueProductDto>   documented response codes: ['410']
Response code counts: { 400: 54, 410: 46 } 


In [293]:
# v1/Tickets
endpoint = endpoints_v1[17]
endpoint.test(100, auth_token)

GET /api/v1/Tickets<(/api/v1/Tickets.get)-query>   documented response codes: ['200', '401']
Response code counts: { 200: 100 } 


In [294]:
# v1/Tickets/useMultiple 
endpoint = endpoints_v1[18]
endpoint.grammar = {
	'<start>': ['/api/v1/Tickets/useMultiple requestBody: <UseMultipleTicketDto>'],
	'<UseMultipleTicketDto>': ['{ "productIds": <productIds.array.integer> }'],
	'<array.integer>': ['<integer>', '<integer>, <array.integer>'],
	'<productIds.array.integer>': ['[]', '[<array.integer>]'],
	'<integer>': ['1', '0', '-1', '4', '10', '11', rand_num]
}
endpoint.test(100, auth_token)

POST /api/v1/Tickets/useMultiple requestBody: <UseMultipleTicketDto>   documented response codes: ['200', '400', '401']
Response code counts: { 200: 53, 400: 47 } 


In [295]:
endpoint = endpoints_v1[19]
endpoint.grammar = {
	'<start>': ['/api/v1/Tickets/use requestBody: <UseTicketDto>'],
	'<UseTicketDto>': ['{ "productId": <productId.integer> }'],
	'<productId.integer>': ['<integer>', '1'],
	'<integer>': ['1', '0', '-1', rand_num, ("10", opts(pre=lambda: random_int_with_bounds(1, 30)))]
}

endpoint.test(100, auth_token)

POST /api/v1/Tickets/use requestBody: <UseTicketDto>   documented response codes: ['200', '400', '401']
Response code counts: { 200: 2, 403: 66, 404: 32 } 
